In [1]:
import spacy
import json
from spacy.tokens import DocBin
from spacy.lang.tr import Turkish
from tqdm import tqdm
from spacy.util import filter_spans
from spacy import displacy
from spacy.tokenizer import Tokenizer
import random
import re
import pickle
import time
import pandas as pd
import numpy as np

C:\Users\melis\anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
f = open('./data/TCDD_annotations.json', encoding='utf-8')
data = json.load(f)

In [307]:
nlp = Turkish()
nlp = spacy.blank('tr')

In [4]:
doc_bin = DocBin()

In [5]:
for example in tqdm(data['annotations']):
    text = example[0]
    labels = example[1]['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("train.spacy")

100%|█████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 678.34it/s]

Skipping entity
Skipping entity


In [8]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2022-10-28 12:14:28.686948: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-28 12:14:28.686996: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-28 12:14:33.601665: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-10-28 12:14:33.601741: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-28 12:14:33.605946: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-VR93MKND
2022-10-28 12:14:33.606055: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-VR93MKND


In [9]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./train.spacy

[i] Saving to output directory: .
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     56.43    3.96   10.53    2.44    0.04
[+] Saved pipeline to output directory
model-last


2022-10-28 12:14:56.372874: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-10-28 12:14:56.372968: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-28 12:15:01.202508: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'nvcuda.dll'; dlerror: nvcuda.dll not found
2022-10-28 12:15:01.202551: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-28 12:15:01.208175: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: LAPTOP-VR93MKND
2022-10-28 12:15:01.208381: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: LAPTOP-VR93MKND
[2022-10-28 12:15:02,152] [INFO] Set up nlp object from config
[2022-10-28 12:15:02,168] [INFO] Pipeline: 

In [2]:
nlp_ner = spacy.load('model-best')

C:\Users\melis\anaconda3\envs\nlp\lib\site-packages\spacy\util.py:877: UserWarning: [W095] Model 'tr_pipeline' (0.0.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.4.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [ ]:
examples = ["15 asker 7 öğretmen 10. ayın 2. haftasının pazartesi gününe Ankara İstanbul bilet almak istiyorum",
           "2 çocuk 28 Ocak'ın ertesi gününe Ankara İstanbul bilet almak istiyorum",
           "3 öğretmen, 15 elma, 7 öğrenci 29 Ekim salı gününe yer sorgulamak istiyorum"]

In [8]:
doc = nlp_ner("2 gün sonra gidip haftaya Eskişehir trenine 2 yetişkin")

colors = {"DURAK": "#F67DE3", "YOLCU": "#7DF6D9", "SAYI":"#a6e22d", "AY":"#FF5733", "GÜN":"#2D14FF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [20]:
#13 ekimde gidip 20'sinde  istanbul ankara hattına 3 bilet almak istiyorum
# engelli, arkadaşım, veli +
# Gazi, şehit yakın, hamile, çocuklu, gazi yakını +
# Haftaya yarın +
# ay, hafta +
# özel günler (yılbaşı, sevgililer günü, bayram) +
# ertesi, ertesi gün +
# virgülsüz gender cümle
# gender <-- hanımefendi, beyefendi, kız +
# kişilik +

# airport soru robotu
#f"{ögün} için {kalkış} {varış} tren biletleri almak istiyorum",
#f"{ögün} kalkışlı {gün} {ay} varışlı {varış} biletleri",

In [2]:
durak_list = ['Ankara', 'İstanbul', 'Eskişehir', 'İzmir', 'Kars', 'Konya', 'Malatya', 'Adana', 'Polatlı', 'Söğütlüçeşme', 'Eryaman', 'Bakırköy', 'Bozüyük', 'Sakarya']
ay_list = ['Ocak', 'Şubat', 'Mart', 'Nisan', 'Mayıs', 'Haziran', 'Temmuz', 'Ağustos', 'Eylül', 'Ekim', 'Kasım', 'Aralık']
weekday_list = ['pazartesi', 'salı', 'çarşamba', 'perşembe', 'cuma', 'cumartesi', 'pazar']
self_list = ['ben', 'bana', 'benim adıma', 'kendim', 'kendime']
specific_delay_list = ['güne', 'gün sonraya', 'gün sonrasına', 'gün ilerisine', 'ay', 'hafta', 'ayın', 'haftasına']
gender_list = ['erkek', 'kadın', 'kız', 'hanımefendi', 'beyefendi']
holidays_list = ['yılbaşı', 'sevgililer günü', 'bayram', 'kurban bayramı', 'ramazan bayramı', 'şeker bayramı', 'kandil', 'arife', 'anneler günü', 'babalar günü', 'öğretmenler günü']
delay_list = ['haftaya', 'günübirlik', 'haftaya yarın', 'ertesi', 'ertesi gün', 'ertesi güne', 'yarın', 'yarına', 'yarın için', 'bugün', 'bugüne', 'haftaya bugün', 'sabaha', 'akşama', 'akşam için', 'sabah için', 'öğleye', 'öğlene', 'öğleden sonraya', 'öğleden sonrasına', 'hafta içi', 'hafta sonu']
yolcu_list = ['yolcu', 'engelli', 'arkadaş', 'askeri personel', 'sakat', 'gazi', 'kişilik', 'kişiye', 'şehit yakını', 'gazi yakını', 'hamile', 'çocuklu', 'arkadaşım', 'veli', 'yetişkin', 'çocuğum', 'yaşlı', 'kişi', 'çocuk', 'hayvan', 'öğrenci', 'genç', 'öğretmen', 'öğretim görevlisi', 'basın', 'muhabir', 'asker', 'personel', 'çalışan', '60 yaş', '65 yaş']

In [117]:
def control_variables(variables_list):
    #split variables
    split_lists = [variable.split() for variable in variables_list]
    splitted_variables = [variable for l in split_lists for variable in l]
    
    for index, var1 in enumerate(splitted_variables):
        for var2 in splitted_variables[index+1:]:
            
            if var1 in var2 or var2 in var1:
                safe = False
                return safe   
    safe = True
    return safe

In [139]:
def control_annotations(annotations):
    for index, annot1 in enumerate(annotations):
        for annot2 in annotations[index+1:]:
            if annot1[:2] == annot2[:2]:
                safe = False
                return safe
    safe = True
    return safe

In [5]:
def tagging(word):
    
    if word in durak_list: tag = 'DURAK'
    if word in ay_list: tag = 'AY'
    if word.isdigit() == True: tag = 'SAYI'
    if word in yolcu_list: tag = 'YOLCU'
    if word in weekday_list: tag = 'GÜN'
    if word in self_list: tag = 'BİREY'
    if word in delay_list: tag = 'ERTELEME'
    if word in specific_delay_list: tag = 'Ö-ERTELEME'
    if word in gender_list: tag = 'CİNSİYET'
    if word in holidays_list: tag = 'ÖZEL-GÜN'
    
    return tag

In [6]:
def find_word(searched_word, selected_sentence):
    splitted_text = selected_sentence.split()
    
    for word in splitted_text:
        
        if searched_word in word and "-" in word:
            start = selected_sentence.find(searched_word)
            end = start + len(searched_word)
            break
        

        elif searched_word in word:
            start = selected_sentence.find(word)
            end = start + len(word)
            break
        
        elif searched_word in selected_sentence:
            start = selected_sentence.find(searched_word)
            end = start + len(searched_word)
            
    return start, end

In [ ]:
# AKTARMALI AKTARMASIZ TREN -
# açık bilet, iade, rezervasyon -

# AİLE, 2 çocuğum ve ben ök + 
# 2 kişilik bilet istiyorum + almak istiyorum
# 2 gün sonraya, 3 güne ök +
# bana, ben, benim adıma, kendim, kendime ök +
# YOLCU_LİST = [PERMİ, PERMİLİ] +
# gidiş dönüş tarihli cümle +

# haftaya, yarın, sabaha, akşama, öğlen, öğleden sonra, haftaya bugün, hafta içi, hafta sonu ök, hafta +
# en yakın, en uygun, en ucuz ök +


In [61]:
ek = {'a':'ya',
      'r':'e',
      'l':'a',
      's':'a',
      'k':'a',
      't':'a',
      'n':'a',
      'z':'a',
      'e':'ye',
      'i':'ye',
      'm':'e',
      'ı':'ya'
     }

In [290]:
ek_dict = {}

In [337]:
with open('ekler', 'wb') as fp:
    pickle.dump(ekler, fp)

In [7]:
with open('ekler', 'rb') as fp:
    ekler = pickle.load(fp)

In [146]:
example_lists = []

In [153]:
for i in range(50):
    s = time.time()
    examples = []
    sentence_index = 0
    while (sentence_index<82):
        count = 0
        while (count<5):
        
            while True:
                kalkış, varış = random.sample(durak_list, 2)
                ay, ay2 = random.sample(ay_list, 2)
                gün, gün2 = random.sample(range(1,32), 2)
                weekday, weekday2 = random.sample(weekday_list, 2)
                yolcu, yolcu2, yolcu3, yolcu4 = random.sample(yolcu_list, 4)
                sayı, sayı2, sayı3, sayı4 = random.sample(range(1,10), 4)
                selff = random.choice(self_list)
                delay = random.choice(delay_list)
                sdelay = random.choice(specific_delay_list)
                gender, gender2 = random.sample(gender_list, 2)
                #ögün = random.choice(holidays_list)

                variables_list = [kalkış, varış, str(gün), str(gün2), ay, ay2, weekday, weekday2, yolcu, yolcu2, yolcu3, yolcu4, str(sayı), str(sayı2), str(sayı3), str(sayı4), delay, sdelay, selff, gender, gender2]

                safe_variables = control_variables(variables_list)
                
                if safe_variables:break
            
            sentence = [f'{kalkış} kalkışlı {varış} varışlı {gün} {ay} treni',
                        f'{gün} {ay} {kalkış} {varış} treni',
                        f'{gün} {ay} {varış} treni {sayı} {yolcu}',
                        f'{kalkış}-{varış} {gün} {ay}',
                        f'{weekday} kalkan {varış} trenleri',
                        f'{weekday} kalkıp {weekday2} varan tren',
                        f'{varış} {gün} {ay}',
                        f'{sayı} {yolcu} {kalkış}-{varış} trenleri',
                        f'{gün} {ay} {kalkış} {varış} tren bileti {sayı} {yolcu}',
                        f'{weekday} kalkış {gün} {ay} dönüş {varış} trenleri',
                        f'{weekday} için {varış} bileti',
                        f'saat {sayı} için {kalkış} {varış} bileti', 
                        f'{gün} {ay} {varış} için bilet',
                        f"{kalkış}'{ekler[kalkış][0]} {varış}'{ekler[varış][1]} bilet",
                        f"{weekday}{ekler[weekday]} {varış} bileti",
                        f"{ay}'{ekler[ay][1]} {gün}'ine {sayı} bilet",
                        f"{kalkış} {varış} {sayı} {yolcu}",
                        f"{gün} {ay}'{ekler[ay][0]} {varış} bileti",
                        f"{sayı} {yolcu} için {kalkış}-{varış} bileti",
                        f"{ay}'{ekler[ay][1]} {gün}'ine {sayı} {varış} bileti",
                        f"{varış}'{ekler[varış][1]} {sayı} {yolcu} bilet istiyorum",
                        f"{selff} ve {yolcu} için {varış} bileti",
                        f"{selff} için {kalkış} {varış} bileti",
                        f"{delay} {sayı} bilet",
                        f"{delay} {kalkış} {varış} bileti",
                        f"{delay} {varış} bileti",
                        f"{delay} {sayı} {yolcu} {kalkış} {varış}",
                        f"{varış} {sayı} {yolcu} {delay}",
                        f"{kalkış} {varış} bileti {delay}",
                        f"{delay} {sayı} bilet istiyorum",
                        f"{sayı} {yolcu} {delay} {kalkış}-{varış} treni",
                        f"{sayı} {sdelay} {kalkış} {varış} bileti {sayı} {yolcu}",
                        f"{sayı} {sdelay} {varış} biletleri",
                        f"{selff} {varış} bileti {sayı} {sdelay}",
                        f"{gün} {ay} gidiş {gün2} {ay2} dönüş {kalkış} {varış} trenleri",
                        f"{gün} {ay} {gün2} {ay2} {varış} bileti {sayı} {yolcu}",
                        f"{gün} {ay} gidiş {gün2} {ay2} dönüş {kalkış} {varış} trenleri",
                        f"{gün} {ay} {gün2} {ay2} {varış} bileti {sayı} {yolcu}",
                        f"{delay} {gün} {ay} dönüşlü {kalkış} {varış} bileti",
                        f"{selff} {delay} {varış} bileti",
                        f"{sayı} {sdelay} {ay} {gün} dönüş tarihli {kalkış}-{varış} bileti almak istiyorum",
                        f"{sayı} {yolcu} ve {selff} {gün} {ay} {kalkış} {varış} bileti aramak istiyorum",
                        f"{ay}'{ekler[ay][1]} {gün}'ine {varış} treni {sayı} {yolcu}",
                        f"{ay}'{ekler[ay][1]} {gün}'i için {kalkış} {varış} bileti",
                        f"{delay} {weekday}ya {kalkış} {varış} tren bileti istiyorun",
                        f"{gün} {ay}'{ekler[ay][2]} {kalkış} {varış} {sayı} bilet almak istiyorum",
                        f"{delay} {weekday} için {sayı} {varış} bileti",
                        f"{ay} ayının {sayı}. haftasında {varış}'a gitmek istiyorum",
                        f"{gün} {ay}'{ekler[ay][2]} gidip {gün2}'inde dönen {varış} treni",
                        f"{weekday} {delay} saat {sayı}'e {kalkış} {varış} bileti",
                        f"{gün}.{sayı}.2023 tarihine {varış} yht bileti istiyorum",
                        f"{gün} {ay} {weekday} günü {sayı} {yolcu} {kalkış}-{varış} tren bileti almak istiyorum",
                        f"{weekday} gününe {sayı} tren bileti",
                        f"{gün} {ay} {weekday} dönüşlü {varış} bileti",
                        f"{weekday} gününe {selff} tren bileti almak istiyorum",
                        f"{gün} {ay}'{ekler[ay][0]} {sayı} adet bilet",
                        f"{gün} {sdelay} {kalkış} {varış} biletlerini göster",
                        f"{varış}",
                        f"{kalkış}-{varış} {delay}",
                        f"{sayı} {yolcu} için {gün} {ay} {weekday}{ekler[weekday]} {varış} biletleri getir",
                        f"{delay} {varış} trenlerini göster",
                        f"{gün} {ay} {sayı} {yolcu} {sayı2} {yolcu2} {kalkış} {varış} tren bileti almak istiyorum",
                        f"{sayı} {yolcu} {gün} {ay} 2023 tarihinde {kalkış} {varış} bileti sorgula",
                        f"{gün} {ay} gününde {sayı} {yolcu}, {sayı2} {yolcu2}, {sayı3} {yolcu3} yer ayırtmak istiyorum",
                        f"{delay} {selff} ve {sayı} {yolcu} için bilet almak istiyorum",
                        f"{sayı} {yolcu} {sayı2} {yolcu2} {delay} {kalkış} {varış} tren sorgulamak istiyorum",
                        f"{selff} ve {sayı} {yolcu} için {kalkış} {varış} boş yer sorgulamak istiyorum",
                        f"{sayı} {yolcu} {sayı2} {yolcu2} {sayı3} {yolcu3} {sayı4} {yolcu4} için {kalkış} {varış} {delay} {weekday} gidiş {gün} {ay} {weekday2} dönüş için boş yer sorgulamak istiyorum",
                        f"{kalkış}-{varış} {delay} {sayı} {yolcu} {sayı2} {yolcu2} {sayı3} {yolcu3} {sayı4} {yolcu4} {weekday} gidiş {gün} {ay} {weekday2} dönüş için {selff} yer sorgulamak istiyorum",
                        f"{sayı} {yolcu} {sayı2} {yolcu2} için {kalkış} {varış} {delay} {weekday} kalkışlı {gün} {ay} dönüşlü {sayı3} {yolcu3} {sayı4} {yolcu4} için bilet sorgulamak istiyorum",
                        f"{kalkış}-{varış} {delay} gidiş {gün} {ay} {weekday} dönmek için {sayı} {yolcu} {sayı2} {yolcu2} {sayı3} {yolcu3} {sayı4} {yolcu4} için rezervasyon istiyorum",
                        f"{kalkış} {varış} {sayı} {sdelay} gidip {gün} {ay} dönen trenler",
                        f"{gün} {ay} tarihine {kalkış}'{ekler[varış][0]} {varış}'{ekler[varış][1]} tren bileti almak istiyorum",
                        f"{selff} ve {yolcu} {kalkış} {varış} tren bileti sorgula",
                        f"{sayı} {gender} {yolcu}, {sayı2} {yolcu2} {gender2} için tren biletleri",
                        f"{sayı} {gender} {yolcu}, {sayı2} {gender2} {varış} tren biletleri sorgula",
                        f"{yolcu} {sayı} {gender}, {yolcu2} {sayı2} {gender2} {kalkış} {varış} tren bileti almak istiyorum",
                        f"{yolcu} {sayı} {gender} {kalkış} {varış} tren bileti istiyoruz",
                        f"{varış} için {gün} {ay} tarihine {sayı} {gender} {yolcu}",
                        f"{sayı} {gender} {yolcu} ve {sayı2} {yolcu2} {gender2} için {kalkış} {varış} {gün} {ay} tren biletleri",
                        f"{sayı} {yolcu} ve {sayı} {yolcu} {varış} tren biletleri {gün} {ay}",
                        f"{sayı} {sdelay} {sayı2} {yolcu} için {kalkış} {varış} tren bileti almak istiyorum",
                        f"{gün} {ay} gidiş {gün2} {ay2} dönüş {kalkış} {varış} treni {sayı} {yolcu}"
                       ]
            
            selected_sentence = sentence[sentence_index]  
            capitalized_sentence = selected_sentence[0].upper() + selected_sentence[1:]
            annotations = []
            for variable in variables_list:
                if variable in selected_sentence:
                    start, end = find_word(variable, selected_sentence)
                    annotation = (start, end, tagging(variable))
                    if annotation not in annotations: 
                        annotations.append(annotation)
            safe_annotations = control_annotations(annotations)
            example = (capitalized_sentence, annotations)
            if example not in examples and safe_annotations: 
                examples.append(example)
                count += 1
         
        sentence_index += 1
    example_lists = example_lists + examples
    e = time.time()

In [154]:
 random.shuffle(example_lists)

In [155]:
len(example_lists)

61500

In [156]:
example_df = pd.DataFrame(example_lists)

In [157]:
example_list2 = example_df.drop_duplicates([0]).values.tolist()

In [162]:
len(example_list2)

51991

In [163]:
with open('annotations', 'wb') as fp:
    pickle.dump(example_list2, fp)

In [164]:
with open('annotations', 'rb') as fp:
    annotations = pickle.load(fp)

In [165]:
annotations

[["3 Kasım'da gidip 20'inde dönen İstanbul treni",
  [(31, 39, 'DURAK'), (0, 1, 'SAYI'), (17, 24, 'SAYI'), (2, 10, 'AY')]],
 ['4 arkadaşım öğleden sonrasına Bozüyük-İzmir treni',
  [(30, 37, 'DURAK'),
   (38, 43, 'DURAK'),
   (2, 11, 'YOLCU'),
   (0, 1, 'SAYI'),
   (12, 29, 'ERTELEME')]],
 ['İzmir', [(0, 5, 'DURAK')]],
 ['5 basın İstanbul-Bakırköy trenleri',
  [(8, 16, 'DURAK'), (17, 25, 'DURAK'), (2, 7, 'YOLCU'), (0, 1, 'SAYI')]],
 ['Ben ve 1 yolcu için Söğütlüçeşme Ankara boş yer sorgulamak istiyorum',
  [(20, 32, 'DURAK'),
   (33, 39, 'DURAK'),
   (9, 14, 'YOLCU'),
   (7, 8, 'SAYI'),
   (0, 3, 'BİREY')]],
 ['4 gazi 3 yolcu için Ankara Kars bugün pazar kalkışlı 11 Ekim dönüşlü 8 engelli 2 yetişkin için bilet sorgulamak istiyorum',
  [(20, 26, 'DURAK'),
   (27, 31, 'DURAK'),
   (53, 55, 'SAYI'),
   (56, 60, 'AY'),
   (38, 43, 'GÜN'),
   (2, 6, 'YOLCU'),
   (9, 14, 'YOLCU'),
   (71, 78, 'YOLCU'),
   (81, 89, 'YOLCU'),
   (0, 1, 'SAYI'),
   (7, 8, 'SAYI'),
   (69, 70, 'SAYI'),
   (79, 8

In [196]:
example = (selected_sentence, kalkış_annotation, varış_annotation, gün_annotation)
print(example)

('16 Ocak İzmir Ankara treni', [(8, 13, 'DURAK')], [(14, 20, 'DURAK')], [(0, 2, 'GÜN')])


In [85]:
nlp_ner = spacy.load('./model-best')

In [86]:
doc = nlp_ner("3 asker 4 yetişkin 7 hayvan 12 çocuk için İstanbul-Ankara haftaya çarşamba gidiş 15 Ekim salı dönüş için boş yer sorgulamak istiyorum")

colors = {"DURAK": "#F67DE3", "YOLCU": "#7DF6D9", "SAYI":"#a6e22d", "AY":"#FF5733", "GÜN":"#2D14FF"}
options = {"colors": colors} 

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)